In [173]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !unzip -qq data.zip

## Imports and Setup

In [2]:
import random
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline

random.seed('2022')

## Load data and EDA/Cleaning


In [ ]:
customers = pd.read_csv('data/customers.csv')
geo = pd.read_csv('data/geo.csv')
transactions = pd.read_csv('data/transactions.csv')

In [107]:
test_ids = transactions[transactions['TEST_SET_ID'].notnull()]['TEST_SET_ID']

In [29]:
customers.dtypes

CUSTOMER                int64
REV_CURRENT_YEAR      float64
REV_CURRENT_YEAR.1    float64
REV_CURRENT_YEAR.2    float64
CREATION_YEAR           int64
OWNERSHIP             float64
COUNTRY                object
dtype: object

In [30]:
customers.head(10)

,CUSTOMER,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,COUNTRY
0,579,138892.876279,138892.876279,153634.810233,19,3.0,CH
1,1656,0.000000,0.000000,767.441860,18,3.0,CH
2,2881,43377.900000,43377.900000,2097.670000,18,3.0,CH
3,2492,82989.850000,82989.850000,83251.950000,19,3.0,CH
4,2391,130.250233,130.250233,3169.084651,19,0.0,CH
5,4478,57116.101538,57116.101538,39577.407692,18,3.0,CH
6,2982,52435.869231,52435.869231,39680.458462,18,3.0,CH
7,3378,863129.080000,863129.080000,552399.830000,19,3.0,CH
8,797,519539.107692,519539.107692,150893.107692,19,3.0,CH
9,2866,128388.234419,128388.234419,19227.570233,19,3.0,CH


In [4]:
customers['REV_CURRENT_YEAR']  = customers['REV_CURRENT_YEAR'].str.replace('"', '')

In [5]:
customers['REV_CURRENT_YEAR'] = customers['REV_CURRENT_YEAR'].astype(str).astype(float)

In [6]:
customers['REV_CURRENT_YEAR'] = np.where(customers['CURRENCY'] == 'Chinese Yuan',
                                         customers['REV_CURRENT_YEAR']*0.14,
                                         np.where(customers['CURRENCY'] == 'Pound Sterling',
                                                  customers['REV_CURRENT_YEAR']*1.20,
                                                  np.where(customers['CURRENCY'] == 'US Dollar',
                                                           customers['REV_CURRENT_YEAR']*0.88,
                                                           customers['REV_CURRENT_YEAR'])))


customers['REV_CURRENT_YEAR.1'] = np.where(customers['CURRENCY'] == 'Chinese Yuan',
                                         customers['REV_CURRENT_YEAR.1']*0.14,
                                         np.where(customers['CURRENCY'] == 'Pound Sterling',
                                                  customers['REV_CURRENT_YEAR.1']*1.20,
                                                  np.where(customers['CURRENCY'] == 'US Dollar',
                                                           customers['REV_CURRENT_YEAR.1']*0.88,
                                                           customers['REV_CURRENT_YEAR.1'])))


customers['REV_CURRENT_YEAR.2'] = np.where(customers['CURRENCY'] == 'Chinese Yuan',
                                         customers['REV_CURRENT_YEAR.2']*0.14,
                                         np.where(customers['CURRENCY'] == 'Pound Sterling',
                                                  customers['REV_CURRENT_YEAR.2']*1.20,
                                                  np.where(customers['CURRENCY'] == 'US Dollar',
                                                           customers['REV_CURRENT_YEAR.2']*0.88,
                                                           customers['REV_CURRENT_YEAR.2'])))

In [7]:
customers = customers.drop(['CURRENCY'], axis=1)
customers['CREATION_YEAR'] = customers['CREATION_YEAR'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
customers['CREATION_YEAR'] = pd.DatetimeIndex(customers['CREATION_YEAR']).year

customers['CREATION_YEAR'] = 2022 - customers['CREATION_YEAR']

In [8]:
customers['COUNTRY'] = np.where(customers['COUNTRY'] == 'Switzerland', 'CH', 'FR')

In [11]:
transactions['CUSTOMER'] = np.where(transactions['CUSTOMER'].str.contains('NA'),
                                   '0', transactions['CUSTOMER'])

In [15]:
transactions['CUSTOMER'] = np.where(transactions['CUSTOMER'].str.contains('#NV'),
                                   '0', transactions['CUSTOMER'])

In [27]:
transactions['CUSTOMER'].unique() 

array([   1,    2,    3, ..., 5980, 5981, 5982])

In [141]:
transactions.dtypes

SO_ID                   object
CUSTOMER                 int64
OFFER_PRICE            float64
SERVICE_LIST_PRICE     float64
MATERIAL_COST          float64
SERVICE_COST           float64
PRICE_LIST             float64
ISIC                   float64
TECH                   float64
OFFER_TYPE             float64
BUSINESS_TYPE          float64
OFFER_STATUS            object
SALES_LOCATION          object
TEST_SET_ID            float64
TOTAL_PRODUCT_COSTS    float64
dtype: object

In [139]:
transactions['TEST_SET_ID'] = pd.to_numeric(transactions['TEST_SET_ID'], downcast='integer')

In [142]:
transactions['TEST_SET_ID'].unique()

array([       nan, 6.0000e+00, 9.0000e+00, ..., 2.6050e+04, 2.6058e+04,
       2.6086e+04])

In [106]:
customers['CUSTOMER'].unique()

array([ 579, 1656, 2881, ..., 6239, 6240, 6241])

In [28]:
transactions.head(10)

,SO_ID,CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,TECH,OFFER_TYPE,BUSINESS_TYPE,OFFER_STATUS,SALES_LOCATION,TEST_SET_ID,TOTAL_PRODUCT_COSTS
0,a030N00001EochoQAB,1,1711.00,1395.0,1107.0,186.30,2.0,2100.0,6.0,22.0,1.0,Lose,Luzern Central,NaN,59.48
1,a030N00001EociNQAR,2,26687.60,14651.0,9282.0,7768.34,1.0,7110.0,1.0,6.0,6.0,Lose,Zürich East,NaN,0.00
2,a030N00001Eod9yQAB,3,6264.70,2296.0,1722.0,2168.56,2.0,6820.0,4.0,19.0,1.0,Win,Luzern Central,NaN,0.00
3,a030N00001EodLEQAZ,4,4300.20,310.0,246.0,2775.92,2.0,3821.0,4.0,19.0,4.0,Win,Basel Central,NaN,0.00
4,a030N00001EoeBiQAJ,5,13693.00,5815.0,4674.0,4179.38,2.0,4719.0,4.0,19.0,1.0,Win,Geneva West,NaN,1854.01
5,a030N00001EoefmQAB,6,23403.95,5932.0,4674.0,15186.39,2.0,6419.0,4.0,19.0,6.0,NaN,Geneva West,6.0,0.00
6,a030N00001EoeovQAB,7,1287.00,930.0,738.0,108.52,2.0,2710.0,4.0,19.0,1.0,Win,Lausanne West,NaN,0.00
7,a030N00001EofDdQAJ,8,8619.60,2310.0,1845.0,3293.22,2.0,4742.0,4.0,19.0,1.0,Win,Zürich East,NaN,0.00
8,a030N00001EofrVQAR,9,52883.55,12075.0,7650.0,22680.97,1.0,8110.0,1.0,7.0,1.0,NaN,Geneva West,9.0,0.00
9,a030N00001Eog9xQAB,10,6485.00,2325.0,1845.0,1857.36,2.0,4321.0,4.0,19.0,1.0,Win,Zürich East,NaN,0.00


In [13]:
transactions['CUSTOMER']  = transactions['CUSTOMER'].str.replace('"', '')
# transactions['CUSTOMER']  = transactions['CUSTOMER'].str.replace('', '')

In [16]:
transactions['CUSTOMER'] = transactions['CUSTOMER'].astype(str).astype(int)

In [17]:
transactions['TOTAL_PRODUCT_COSTS'] = transactions['COSTS_PRODUCT_A'] + transactions['COSTS_PRODUCT_B'] + transactions['COSTS_PRODUCT_C'] + transactions['COSTS_PRODUCT_D'] + transactions['COSTS_PRODUCT_E']

In [18]:
transactions['TOTAL_PRODUCT_COSTS'] = transactions['TOTAL_PRODUCT_COSTS'] + transactions['COSTS_PRODUCT_E']

In [19]:
transactions = transactions.drop(['MO_ID', 
                            'END_CUSTOMER',
                            'MO_CREATED_DATE',
                            'SO_CREATED_DATE',
                            'COSTS_PRODUCT_A',
                            'COSTS_PRODUCT_B',
                            'COSTS_PRODUCT_C',
                            'COSTS_PRODUCT_D',
                            'COSTS_PRODUCT_E'], axis = 1)

In [20]:
Win = ['WIN', 'Won', 'WON']
Lose = ['Lost', 'LOST', 'LOsT']

In [21]:
transactions['OFFER_STATUS'] = np.where(transactions['OFFER_STATUS'].isin(Lose),
                                     'Lose', transactions['OFFER_STATUS'])
transactions['OFFER_STATUS'] = np.where(transactions['OFFER_STATUS'].isin(Win),
                                     'Win', transactions['OFFER_STATUS'])

## Encode categorical features

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

In [24]:
transactions['PRICE_LIST'] = encoder.fit_transform(transactions['PRICE_LIST'].values.reshape(-1, 1))
transactions['TECH'] = encoder.fit_transform(transactions['TECH'].values.reshape(-1, 1))
transactions['OFFER_TYPE'] = encoder.fit_transform(transactions['OFFER_TYPE'].values.reshape(-1, 1))      
transactions['BUSINESS_TYPE'] = encoder.fit_transform(transactions['BUSINESS_TYPE'].values.reshape(-1, 1))

In [25]:
customers['OWNERSHIP'] = encoder.fit_transform(customers['OWNERSHIP'].values.reshape(-1, 1))

In [31]:
geo.dtypes

COUNTRY           object
SALES_OFFICE      object
SALES_BRANCH      object
SALES_LOCATION    object
dtype: object

In [32]:
geo.head(10)

,COUNTRY,SALES_OFFICE,SALES_BRANCH,SALES_LOCATION
0,CH,Luzern,Branch Central,Luzern Central
1,CH,Zürich,Branch East,Zürich East
2,CH,Basel,Branch Central,Basel Central
3,CH,Geneva,Branch West,Geneva West
4,CH,Lausanne,Branch West,Lausanne West
5,CH,Sion,Branch West,Sion West
6,CH,St. Gallen,Branch East,St. Gallen East
7,CH,St. Blaise,Branch West,St. Blaise West
8,CH,Steinhausen,Branch Central,Steinhausen Central
9,CH,Bern,Branch Central,Bern Central


## Merge cleaned data frames

In [33]:
df_transactions_geo = pd.merge(geo, transactions, on='SALES_LOCATION', how='left')

In [138]:
df_transactions_geo['TEST_SET_ID'].unique()

array([   nan,   526.,   733., ..., 23180., 24938., 25202.])

In [34]:
df_transactions_geo.head(10)

,COUNTRY,SALES_OFFICE,SALES_BRANCH,SALES_LOCATION,SO_ID,CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,TECH,OFFER_TYPE,BUSINESS_TYPE,OFFER_STATUS,TEST_SET_ID,TOTAL_PRODUCT_COSTS
0,CH,Luzern,Branch Central,Luzern Central,a030N00001EochoQAB,1.0,1711.0,1395.00,1107.0,186.30,2.0,2100.0,6.0,22.0,1.0,Lose,NaN,59.48
1,CH,Luzern,Branch Central,Luzern Central,a030N00001Eod9yQAB,3.0,6264.7,2296.00,1722.0,2168.56,2.0,6820.0,4.0,19.0,1.0,Win,NaN,0.00
2,CH,Luzern,Branch Central,Luzern Central,a030N00001HJxEoQAL,32.0,1291.6,775.00,615.0,281.03,2.0,4652.0,6.0,28.0,1.0,Win,NaN,0.00
3,CH,Luzern,Branch Central,Luzern Central,a030N00001HKmkwQAD,57.0,7879.0,4050.00,3321.0,2033.37,2.0,8299.0,6.0,22.0,4.0,Win,NaN,1755.30
4,CH,Luzern,Branch Central,Luzern Central,a030N00001HKufKQAT,82.0,20000.0,11753.00,7446.0,8591.00,1.0,4923.0,1.0,25.0,4.0,Win,NaN,0.00
5,CH,Luzern,Branch Central,Luzern Central,a030N00001HKw8BQAT,89.0,50744.0,16583.00,10506.0,3747.44,1.0,2816.0,1.0,7.0,4.0,Win,NaN,0.00
6,CH,Luzern,Branch Central,Luzern Central,a030N00001HKXYvQAP,101.0,1818.0,1085.00,861.0,368.62,2.0,2011.0,4.0,19.0,4.0,Win,NaN,368.62
7,CH,Luzern,Branch Central,Luzern Central,a030N00001HL4IrQAL,106.0,7030.0,3720.00,2952.0,1915.87,2.0,NaN,6.0,22.0,4.0,Win,NaN,1398.19
8,CH,Luzern,Branch Central,Luzern Central,a030N00001HmRtlQAF,116.0,22209.0,12425.00,9786.0,7848.88,2.0,8610.0,4.0,19.0,4.0,Win,NaN,7155.59
9,CH,Luzern,Branch Central,Luzern Central,a030N00001HXvd4QAD,123.0,10059.3,3018.75,1912.5,4608.67,1.0,4322.0,1.0,27.0,4.0,Win,NaN,0.00


In [35]:
df_all = pd.merge(df_transactions_geo, customers, on=['CUSTOMER', 'COUNTRY'], how='left')

In [36]:
df_all.shape

(26115, 23)

In [120]:
df_all.isnull().sum()

COUNTRY                    0
SALES_OFFICE               1
SALES_BRANCH               0
SALES_LOCATION             0
SO_ID                      0
CUSTOMER                   0
OFFER_PRICE                0
SERVICE_LIST_PRICE         0
MATERIAL_COST              0
SERVICE_COST               0
PRICE_LIST                 0
ISIC                       0
TECH                       0
OFFER_TYPE                 0
BUSINESS_TYPE              0
OFFER_STATUS            2574
TEST_SET_ID            23540
TOTAL_PRODUCT_COSTS        0
REV_CURRENT_YEAR           0
REV_CURRENT_YEAR.1         0
REV_CURRENT_YEAR.2         0
CREATION_YEAR              0
OWNERSHIP                  0
dtype: int64

In [121]:
df_all.shape

(26114, 23)

In [118]:
df_all[df_all['SO_ID'].isnull()]

,COUNTRY,SALES_OFFICE,SALES_BRANCH,SALES_LOCATION,SO_ID,CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,...,OFFER_TYPE,BUSINESS_TYPE,OFFER_STATUS,TEST_SET_ID,TOTAL_PRODUCT_COSTS,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP
17028,CH,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,340343.98284,340343.98284,353286.479937,15.580069,2.515583


In [119]:
df_all = df_all.drop(labels=17028, axis=0)

In [115]:
df_all['ISIC']= df_all['ISIC'].fillna(df_all['ISIC'].mean())
df_all['REV_CURRENT_YEAR']= df_all['REV_CURRENT_YEAR'].fillna(df_all['REV_CURRENT_YEAR'].mean())
df_all['REV_CURRENT_YEAR.1']= df_all['REV_CURRENT_YEAR.1'].fillna(df_all['REV_CURRENT_YEAR.1'].mean())
df_all['REV_CURRENT_YEAR.2']= df_all['REV_CURRENT_YEAR.2'].fillna(df_all['REV_CURRENT_YEAR.2'].mean())
df_all['CREATION_YEAR']= df_all['CREATION_YEAR'].fillna(df_all['CREATION_YEAR'].mean())
df_all['OWNERSHIP']= df_all['OWNERSHIP'].fillna(df_all['OWNERSHIP'].mean())

In [122]:
df_all.head(10)

,COUNTRY,SALES_OFFICE,SALES_BRANCH,SALES_LOCATION,SO_ID,CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,...,OFFER_TYPE,BUSINESS_TYPE,OFFER_STATUS,TEST_SET_ID,TOTAL_PRODUCT_COSTS,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP
0,CH,Luzern,Branch Central,Luzern Central,a030N00001EochoQAB,1.0,1711.0,1395.00,1107.0,186.30,...,22.0,1.0,Lose,NaN,59.48,1.137975e+04,1.137975e+04,4.508506e+03,18.000000,3.000000
1,CH,Luzern,Branch Central,Luzern Central,a030N00001Eod9yQAB,3.0,6264.7,2296.00,1722.0,2168.56,...,19.0,1.0,Win,NaN,0.00,3.403440e+05,3.403440e+05,3.532865e+05,15.580069,2.515583
2,CH,Luzern,Branch Central,Luzern Central,a030N00001HJxEoQAL,32.0,1291.6,775.00,615.0,281.03,...,28.0,1.0,Win,NaN,0.00,1.537480e+05,1.537480e+05,1.198337e+05,19.000000,3.000000
3,CH,Luzern,Branch Central,Luzern Central,a030N00001HKmkwQAD,57.0,7879.0,4050.00,3321.0,2033.37,...,22.0,4.0,Win,NaN,1755.30,3.967880e+05,3.967880e+05,5.980782e+05,18.000000,0.000000
4,CH,Luzern,Branch Central,Luzern Central,a030N00001HKufKQAT,82.0,20000.0,11753.00,7446.0,8591.00,...,25.0,4.0,Win,NaN,0.00,3.662662e+05,3.662662e+05,6.172848e+05,19.000000,3.000000
5,CH,Luzern,Branch Central,Luzern Central,a030N00001HKw8BQAT,89.0,50744.0,16583.00,10506.0,3747.44,...,7.0,4.0,Win,NaN,0.00,7.837889e+05,7.837889e+05,9.640110e+05,19.000000,3.000000
6,CH,Luzern,Branch Central,Luzern Central,a030N00001HKXYvQAP,101.0,1818.0,1085.00,861.0,368.62,...,19.0,4.0,Win,NaN,368.62,2.637720e+03,2.637720e+03,9.047900e+02,13.000000,3.000000
7,CH,Luzern,Branch Central,Luzern Central,a030N00001HL4IrQAL,106.0,7030.0,3720.00,2952.0,1915.87,...,22.0,4.0,Win,NaN,1398.19,1.676505e+04,1.676505e+04,2.293086e+03,18.000000,3.000000
8,CH,Luzern,Branch Central,Luzern Central,a030N00001HmRtlQAF,116.0,22209.0,12425.00,9786.0,7848.88,...,19.0,4.0,Win,NaN,7155.59,1.371694e+06,1.371694e+06,1.697032e+06,19.000000,3.000000
9,CH,Luzern,Branch Central,Luzern Central,a030N00001HXvd4QAD,123.0,10059.3,3018.75,1912.5,4608.67,...,27.0,4.0,Win,NaN,0.00,3.787316e+04,3.787316e+04,2.225315e+04,18.000000,3.000000


In [134]:
df_all.to_csv("all_data.csv")

In [123]:
train = df_all[df_all['TEST_SET_ID'].isna()]
test  = df_all[df_all['TEST_SET_ID'].notnull()]

In [124]:
test_ids_wrong = df_all[df_all['TEST_SET_ID'].notnull()]['TEST_SET_ID']

In [125]:
test_ids = pd.to_numeric(test_ids, downcast='integer')
test_ids_wrong = pd.to_numeric(test_ids_wrong, downcast='integer')

In [126]:
test_ids

5            6
8            9
13          14
34          35
35          36
         ...  
26018    26019
26032    26033
26049    26050
26057    26058
26085    26086
Name: TEST_SET_ID, Length: 2576, dtype: int16

In [127]:
test_ids_wrong

37         526
53         733
57         839
63         960
94        1537
         ...  
26083    21852
26088    22490
26093    23180
26104    24938
26107    25202
Name: TEST_SET_ID, Length: 2574, dtype: int16

In [128]:
train = train.drop(['TEST_SET_ID', 'CUSTOMER'], axis = 1)
test = test.drop(['TEST_SET_ID', 'CUSTOMER'], axis = 1)

In [129]:
test.head(10)

,COUNTRY,SALES_OFFICE,SALES_BRANCH,SALES_LOCATION,SO_ID,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,...,TECH,OFFER_TYPE,BUSINESS_TYPE,OFFER_STATUS,TOTAL_PRODUCT_COSTS,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP
37,CH,Luzern,Branch Central,Luzern Central,a030N00001JVd54QAD,1026.0,930.0,738.0,0.00,2.0,...,6.0,0.0,1.0,NaN,0.00,2.274454e+04,2.274454e+04,1.057334e+05,10.000000,3.000000
53,CH,Luzern,Branch Central,Luzern Central,a030N00001JWTexQAH,6482.0,4437.0,3321.0,794.45,2.0,...,4.0,19.0,1.0,NaN,794.45,5.977609e+06,5.977609e+06,5.035376e+06,16.000000,3.000000
57,CH,Luzern,Branch Central,Luzern Central,a030N00001JXiB5QAL,6763.0,2790.0,2274.0,1986.62,2.0,...,6.0,22.0,6.0,NaN,490.52,3.403440e+05,3.403440e+05,3.532865e+05,15.580069,2.515583
63,CH,Luzern,Branch Central,Luzern Central,a030N00001LvqVoQAJ,1944.0,1671.0,1131.0,160.52,2.0,...,4.0,19.0,1.0,NaN,0.00,8.451643e+03,8.451643e+03,0.000000e+00,18.000000,3.000000
94,CH,Luzern,Branch Central,Luzern Central,a030N00001OPTWpQAP,34129.7,10740.0,6120.0,10219.14,1.0,...,1.0,6.0,6.0,NaN,0.00,2.914823e+05,2.914823e+05,1.735712e+05,18.000000,3.000000
95,CH,Luzern,Branch Central,Luzern Central,a030N00001OQ074QAD,6403.0,4714.0,3016.0,876.64,2.0,...,4.0,19.0,1.0,NaN,0.00,0.000000e+00,0.000000e+00,0.000000e+00,18.000000,3.000000
178,CH,Luzern,Branch Central,Luzern Central,a030N00001Isv2kQAB,4905.0,4495.0,3567.0,85.61,2.0,...,6.0,22.0,1.0,NaN,85.61,1.140013e+05,1.140013e+05,1.031970e+05,19.000000,3.000000
192,CH,Luzern,Branch Central,Luzern Central,a030N00001Iu64zQAB,1909.0,1479.0,1107.0,171.60,2.0,...,4.0,19.0,1.0,NaN,0.00,3.403440e+05,3.403440e+05,3.532865e+05,15.580069,2.515583
200,CH,Luzern,Branch Central,Luzern Central,a030N00001JVliNQAT,2501.1,1985.0,1611.0,57.73,2.0,...,4.0,19.0,1.0,NaN,750.00,5.977609e+06,5.977609e+06,5.035376e+06,16.000000,3.000000
206,CH,Luzern,Branch Central,Luzern Central,a030N00001IuAcEQAV,16455.0,7728.0,4896.0,4053.00,1.0,...,1.0,6.0,4.0,NaN,0.00,2.914823e+05,2.914823e+05,1.735712e+05,18.000000,3.000000


In [46]:
test = test.drop(['SO_ID'], axis=1)

In [47]:
train = train.drop(['SO_ID'], axis=1)

In [133]:
test.isnull().sum()

OFFER_PRICE               0
SERVICE_LIST_PRICE        0
MATERIAL_COST             0
SERVICE_COST              0
PRICE_LIST                0
ISIC                      0
TECH                      0
OFFER_TYPE                0
BUSINESS_TYPE             0
OFFER_STATUS           2574
TOTAL_PRODUCT_COSTS       0
REV_CURRENT_YEAR          0
REV_CURRENT_YEAR.1        0
REV_CURRENT_YEAR.2        0
CREATION_YEAR             0
OWNERSHIP                 0
dtype: int64

In [131]:
train = train.drop(['COUNTRY',
            'SALES_OFFICE',
           'SALES_BRANCH',
           'SALES_LOCATION'], axis=1)

In [132]:
test = test.drop(['COUNTRY',
                  'SO_ID',
            'SALES_OFFICE',
           'SALES_BRANCH',
           'SALES_LOCATION'], axis=1)

In [52]:
test = test.drop(['OFFER_STATUS'], axis=1)

In [53]:
train.head(10)

,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,TECH,OFFER_TYPE,BUSINESS_TYPE,OFFER_STATUS,TOTAL_PRODUCT_COSTS,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP
0,1711.0,1395.00,1107.0,186.30,2.0,2100.0,6.0,22.0,1.0,Lose,59.48,1.137975e+04,1.137975e+04,4.508506e+03,18.0,3.0
1,6264.7,2296.00,1722.0,2168.56,2.0,6820.0,4.0,19.0,1.0,Win,0.00,NaN,NaN,NaN,NaN,NaN
2,1291.6,775.00,615.0,281.03,2.0,4652.0,6.0,28.0,1.0,Win,0.00,1.537480e+05,1.537480e+05,1.198337e+05,19.0,3.0
3,7879.0,4050.00,3321.0,2033.37,2.0,8299.0,6.0,22.0,4.0,Win,1755.30,3.967880e+05,3.967880e+05,5.980782e+05,18.0,0.0
4,20000.0,11753.00,7446.0,8591.00,1.0,4923.0,1.0,25.0,4.0,Win,0.00,3.662662e+05,3.662662e+05,6.172848e+05,19.0,3.0
5,50744.0,16583.00,10506.0,3747.44,1.0,2816.0,1.0,7.0,4.0,Win,0.00,7.837889e+05,7.837889e+05,9.640110e+05,19.0,3.0
6,1818.0,1085.00,861.0,368.62,2.0,2011.0,4.0,19.0,4.0,Win,368.62,2.637720e+03,2.637720e+03,9.047900e+02,13.0,3.0
7,7030.0,3720.00,2952.0,1915.87,2.0,NaN,6.0,22.0,4.0,Win,1398.19,1.676505e+04,1.676505e+04,2.293086e+03,18.0,3.0
8,22209.0,12425.00,9786.0,7848.88,2.0,8610.0,4.0,19.0,4.0,Win,7155.59,1.371694e+06,1.371694e+06,1.697032e+06,19.0,3.0
9,10059.3,3018.75,1912.5,4608.67,1.0,4322.0,1.0,27.0,4.0,Win,0.00,3.787316e+04,3.787316e+04,2.225315e+04,18.0,3.0


In [54]:
train['OFFER_STATUS'] = np.where(train['OFFER_STATUS'] == 'Win', 1, 0)

In [ ]:
# test['OFFER_STATUS'] = np.where(test['OFFER_STATUS'] == 'Win', 1, 0)

In [64]:
train.isnull().sum()

OFFER_PRICE            0
SERVICE_LIST_PRICE     0
MATERIAL_COST          0
SERVICE_COST           0
PRICE_LIST             0
ISIC                   0
TECH                   0
OFFER_TYPE             0
BUSINESS_TYPE          0
OFFER_STATUS           0
TOTAL_PRODUCT_COSTS    0
REV_CURRENT_YEAR       0
REV_CURRENT_YEAR.1     0
REV_CURRENT_YEAR.2     0
CREATION_YEAR          0
OWNERSHIP              0
dtype: int64

In [63]:
# train[train['SO_ID'].isnull()]
# train = train.drop(train.index[17028])
# train = train.drop(labels=17028, axis=0)

In [59]:
# train['ISIC']= train['ISIC'].fillna(train['ISIC'].mean())
# train['REV_CURRENT_YEAR']= train['REV_CURRENT_YEAR'].fillna(train['REV_CURRENT_YEAR'].mean())
# train['REV_CURRENT_YEAR.1']= train['REV_CURRENT_YEAR.1'].fillna(train['REV_CURRENT_YEAR.1'].mean())
# train['REV_CURRENT_YEAR.2']= train['REV_CURRENT_YEAR.2'].fillna(train['REV_CURRENT_YEAR.2'].mean())
# train['CREATION_YEAR']= train['CREATION_YEAR'].fillna(train['CREATION_YEAR'].mean())
# train['OWNERSHIP']= train['OWNERSHIP'].fillna(train['OWNERSHIP'].mean())

In [57]:
# test['ISIC']= test['ISIC'].fillna(test['ISIC'].mean())
# test['REV_CURRENT_YEAR']= test['REV_CURRENT_YEAR'].fillna(test['REV_CURRENT_YEAR'].mean())
# test['REV_CURRENT_YEAR.1']= test['REV_CURRENT_YEAR.1'].fillna(test['REV_CURRENT_YEAR.1'].mean())
# test['REV_CURRENT_YEAR.2']= test['REV_CURRENT_YEAR.2'].fillna(test['REV_CURRENT_YEAR.2'].mean())
# test['CREATION_YEAR']= test['CREATION_YEAR'].fillna(test['CREATION_YEAR'].mean())
# test['OWNERSHIP']= test['OWNERSHIP'].fillna(test['OWNERSHIP'].mean())

In [66]:
target = train['OFFER_STATUS']
train = train.drop(['OFFER_STATUS'], axis=1)

## Data splits for training

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.20, random_state=8, stratify=target)

In [68]:
X_train.shape, X_val.shape

((18832, 15), (4708, 15))

In [97]:
y_train

19618    1
17672    1
18005    0
21603    1
18817    1
        ..
41       1
10950    0
21567    1
5182     1
10686    1
Name: OFFER_STATUS, Length: 18832, dtype: int64

In [93]:
X_train.dtypes

OFFER_PRICE            float64
SERVICE_LIST_PRICE     float64
MATERIAL_COST          float64
SERVICE_COST           float64
PRICE_LIST             float64
ISIC                   float64
TECH                   float64
OFFER_TYPE             float64
BUSINESS_TYPE          float64
TOTAL_PRODUCT_COSTS    float64
REV_CURRENT_YEAR       float64
REV_CURRENT_YEAR.1     float64
REV_CURRENT_YEAR.2     float64
CREATION_YEAR          float64
OWNERSHIP              float64
dtype: object

In [71]:
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

cols = X_train.columns

In [72]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_val = scaler.transform(X_val)

In [73]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [74]:
X_val = pd.DataFrame(X_val, columns=[cols])

## Model Training

In [75]:
from sklearn.svm import SVC


# import metrics to compute accuracy
from sklearn.metrics import accuracy_score


# instantiate classifier with default hyperparameters
svc=SVC() 


# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_val)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_val, y_pred)))

Model accuracy score with default hyperparameters: 0.8101


## Generate predictions

In [102]:
y_train_pred = svc.predict(X_train)

In [103]:
print(accuracy_score(y_train, y_train_pred))

0.818447323704333


In [104]:
test.head(10)

,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,TECH,OFFER_TYPE,BUSINESS_TYPE,TOTAL_PRODUCT_COSTS,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP
37,1026.0,930.0,738.0,0.00,2.0,6619.000000,6.0,0.0,1.0,0.00,2.274454e+04,2.274454e+04,1.057334e+05,10.00000,3.000000
53,6482.0,4437.0,3321.0,794.45,2.0,8110.000000,4.0,19.0,1.0,794.45,5.977609e+06,5.977609e+06,5.035376e+06,16.00000,3.000000
57,6763.0,2790.0,2274.0,1986.62,2.0,8422.000000,6.0,22.0,6.0,490.52,1.432479e+06,1.432479e+06,1.493724e+06,16.85115,2.298507
63,1944.0,1671.0,1131.0,160.52,2.0,7110.000000,4.0,19.0,1.0,0.00,8.451643e+03,8.451643e+03,0.000000e+00,18.00000,3.000000
94,34129.7,10740.0,6120.0,10219.14,1.0,4322.000000,1.0,6.0,6.0,0.00,2.914823e+05,2.914823e+05,1.735712e+05,18.00000,3.000000
95,6403.0,4714.0,3016.0,876.64,2.0,4321.000000,4.0,19.0,1.0,0.00,0.000000e+00,0.000000e+00,0.000000e+00,18.00000,3.000000
178,4905.0,4495.0,3567.0,85.61,2.0,4652.000000,6.0,22.0,1.0,85.61,1.140013e+05,1.140013e+05,1.031970e+05,19.00000,3.000000
192,1909.0,1479.0,1107.0,171.60,2.0,5581.624951,4.0,19.0,1.0,0.00,1.432479e+06,1.432479e+06,1.493724e+06,16.85115,2.298507
200,2501.1,1985.0,1611.0,57.73,2.0,8110.000000,4.0,19.0,1.0,750.00,5.977609e+06,5.977609e+06,5.035376e+06,16.00000,3.000000
206,16455.0,7728.0,4896.0,4053.00,1.0,4322.000000,1.0,6.0,4.0,0.00,2.914823e+05,2.914823e+05,1.735712e+05,18.00000,3.000000


In [105]:
y_test_pred=svc.predict(test)

In [106]:
y_test_pred

array([1, 1, 1, ..., 1, 1, 1])

In [81]:
type(test_ids)

numpy.ndarray

## Prepare submission file

In [ ]:
test_ids = test_ids.to_numpy()
test_ids
d = {'id': test_ids, 'prediction':y_test_pred}
d
a = pd.DataFrame(d)
a.to_csv("foo.csv")

In [85]:
a.head(10)

,id,prediction
0,526,1
1,733,1
2,839,1
3,960,1
4,1537,1
5,1557,1
6,2976,1
7,3149,1
8,3335,1
9,3475,1


## Additional Model Testing using scikit-learn

In [351]:
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils.extmath import density
from sklearn import metrics
from time import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder

In [352]:
models = {
    "Ridge_Classifier": RidgeClassifier(tol=1e-2, solver="auto"),
    "Perceptron": Perceptron(),
    "Passive_Aggressive": PassiveAggressiveClassifier(),
    "kNN": KNeighborsClassifier(n_neighbors=10),
    "Random_forest": RandomForestClassifier(n_estimators=100),
    "LinearSVC_l1": LinearSVC(penalty="l1", dual=False, tol=1e-3),
    "LinearSVC_l2": LinearSVC(penalty="l2", dual=False, tol=1e-3),
    "SGDClassifier_l1": SGDClassifier(alpha=.0001, penalty="l1"),
    "SGDClassifier_l2": SGDClassifier(alpha=.0001, penalty="l2"),
    "SGDClassifier_elasticnet": SGDClassifier(alpha=.0001, penalty="elasticnet"),
    "NearestCentroid": NearestCentroid(),
    "MultinomialNB": MultinomialNB(alpha=.01),
    "BernoulliNB": BernoulliNB(alpha=.01),
    "LinearSVC_L1_based_feature_selection": Pipeline([
                              ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False,tol=1e-3))),
                              ('classification', LinearSVC(penalty="l2"))])    
}


In [353]:
def benchmark(clf, X_train, X_test, y_train, y_test):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.f1_score(y_test, pred, average='weighted')
    print("score:   %0.3f" % score)

    acc = balanced_accuracy_score(y_val, pred)
    
    

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, acc

In [ ]:
results_all = []
for model_name, model in models.items():
    clf_descr, score, acc = benchmark(model, X_train, X_val, y_train, y_val)
    results_all.append({
        'model_name': model_name,
        'score': score,
        'acc': acc            
    })

## Experimental tests with PyTorch Tabular

In [249]:
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
clf1_nopreproc = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )

# fit the model 
clf1_nopreproc.fit(
    x_train,y_train,
    eval_set=[(x_train, y_train), (x_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['auc','accuracy'],
    max_epochs=1000 , patience=50,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)            

In [ ]:
transactions = pd.merge(transactions, geo, how="left", on="SALES_LOCATION")

In [ ]:
transactions['Country'] = np.where(transactions['COUNTRY'] == 'CH', 'Switzerland', 'France')

In [ ]:
customers['CUSTOMER'] = customers['CUSTOMER'].map(str)  + '_' + customers['COUNTRY']
transactions['CUSTOMER']  = transactions['CUSTOMER'].str.replace('"', '')
#transactions['CUSTOMER']  = pd.to_numeric(transactions['CUSTOMER'], errors='coerce', downcast='integer')
transactions['CUSTOMER'] = transactions['CUSTOMER'].map(str)  + '_' + transactions['Country']

In [ ]:
len(transactions)

26151

In [ ]:
customers.head(2)

,CUSTOMER,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,COUNTRY,CURRENCY
0,579_Switzerland,"""157832.813953488""",157832.813953,174585.011628,01/01/2003,Privately Owned/Publicly Traded,Switzerland,US Dollar
1,1656_Switzerland,"""0""",0.000000,872.093023,01.01.2004,Privately Owned/Publicly Traded,Switzerland,US Dollar


In [ ]:
transactions['MO_CREATED_DATE'] = transactions['MO_CREATED_DATE'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
transactions['SO_CREATED_DATE'] = transactions['SO_CREATED_DATE'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
customers['CREATION_YEAR'] = customers['CREATION_YEAR'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))

In [ ]:
transactions['MO_CREATED_DATE'] = pd.DatetimeIndex(transactions['MO_CREATED_DATE']).year
transactions['SO_CREATED_DATE'] = pd.DatetimeIndex(transactions['SO_CREATED_DATE']).year
customers['CREATION_YEAR'] = pd.DatetimeIndex(customers['CREATION_YEAR']).year

In [ ]:
#geo.head(100)

In [ ]:
#transactions.head(2)

In [ ]:
#transactions.drop(transactions.index[transactions["CUSTOMER"].apply(lambda x: not (x.strip().isnumeric()))], axis=0, inplace=True)

In [ ]:
print(len(transactions), len(customers))

26151 8452


In [ ]:
df_merged = pd.merge(transactions, customers, how="left", on="CUSTOMER")

In [ ]:
list(df_merged.columns)

['MO_ID',
 'SO_ID',
 'CUSTOMER',
 'END_CUSTOMER',
 'OFFER_PRICE',
 'SERVICE_LIST_PRICE',
 'MATERIAL_COST',
 'SERVICE_COST',
 'PRICE_LIST',
 'ISIC',
 'MO_CREATED_DATE',
 'SO_CREATED_DATE',
 'TECH',
 'OFFER_TYPE',
 'BUSINESS_TYPE',
 'COSTS_PRODUCT_A',
 'COSTS_PRODUCT_B',
 'COSTS_PRODUCT_C',
 'OFFER_STATUS',
 'COSTS_PRODUCT_D',
 'COSTS_PRODUCT_E',
 'SALES_LOCATION',
 'TEST_SET_ID',
 'COUNTRY_x',
 'SALES_OFFICE',
 'SALES_BRANCH',
 'Country',
 'REV_CURRENT_YEAR',
 'REV_CURRENT_YEAR.1',
 'REV_CURRENT_YEAR.2',
 'CREATION_YEAR',
 'OWNERSHIP',
 'COUNTRY_y',
 'CURRENCY']

In [ ]:
#missing = df_merged[(df_merged['REV_CURRENT_YEAR'].isnull()) & (df_merged['TEST_SET_ID'].notnull())]

In [ ]:
# len(missing)

95

In [ ]:
# missing

,MO_ID,SO_ID,CUSTOMER,END_CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,MO_CREATED_DATE,SO_CREATED_DATE,TECH,OFFER_TYPE,BUSINESS_TYPE,COSTS_PRODUCT_A,COSTS_PRODUCT_B,COSTS_PRODUCT_C,OFFER_STATUS,COSTS_PRODUCT_D,COSTS_PRODUCT_E,SALES_LOCATION,TEST_SET_ID,COUNTRY_x,SALES_OFFICE,SALES_BRANCH,Country,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,COUNTRY_y,CURRENCY
300,a050N00001CwHg4QAF,a030N00001I3HWMQA3,265_Switzerland,NaN,8664.25,1860.00,1476.00,5400.00,SFT Standard,8422.0,2019,2019,S,IN,M,0.0,0.0,0.0,NaN,0.0,0.00,Bern Central,301.0,CH,Bern,Branch Central,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,a050N00001D9F2iQAF,a030N00001IssweQAB,270_Switzerland,NaN,19547.60,3220.00,2040.00,11720.00,CMT Installer,2652.0,2019,2019,C,D,E,0.0,0.0,0.0,NaN,0.0,0.00,St. Blaise West,306.0,CH,St. Blaise,Branch West,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,a050N00001DhU6jQAF,a030N00001ItFMjQAN,306_Switzerland,NaN,7032.90,1240.00,984.00,3480.00,SFT Standard,2100.0,2019,2019,S,V,E,0.0,0.0,0.0,NaN,0.0,0.00,Lausanne West,358.0,CH,Lausanne,Branch West,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,a050N00001HPx0fQAD,a030N00001Jw2iGQAR,270_Switzerland,NaN,17086.95,9633.00,5814.00,7160.00,CMT End Customer,2652.0,2019,2019,C,D,E,0.0,0.0,0.0,NaN,0.0,0.00,St. Blaise West,641.0,CH,St. Blaise,Branch West,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,a050N00001FfLMDQA3,a030N00001JWKptQAH,306_Switzerland,NaN,1483.25,1490.00,1230.00,11.17,SFT Standard,2100.0,2019,2019,S,IN,E,0.0,0.0,0.0,NaN,0.0,0.00,Lausanne West,709.0,CH,Lausanne,Branch West,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24423,a050N00001HQGkxQAH,a030N00001JwfnKQAR,NA_France,NaN,35600.00,22945.71,14941.20,9956.22,Tarif public,7110.0,2019,2019,F,FDD,Mig,0.0,0.0,0.0,NaN,0.0,0.00,Bordeaux Sud Ouest,24424.0,FR,Bordeaux,Sud Ouest,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24455,a050N00001FfcQFQAZ,a030N00001JWQDjQAP,NA_France,NaN,1116.40,772.00,464.24,154.70,Tarif public,4321.0,2019,2019,S,SU,Exp,0.0,0.0,0.0,NaN,0.0,0.00,Nantes Ouest,24456.0,FR,Nantes,Ouest,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24869,a050N00001L0LaCQAV,a030N00001OMdZbQAL,NA_France,NaN,19931.19,3662.00,2449.80,17396.12,Tarif public,4321.0,2020,2020,F,FDI,Mig,0.0,0.0,0.0,NaN,0.0,2074.60,Clermont Centre-Est,24870.0,FR,Clermont,Centre-Est,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25063,a050N00001HRGxJQAX,a030N00001LvoEYQAZ,NA_France,NaN,1438.70,1102.00,727.40,273.36,Tarif public,NaN,2019,2019,F,FDI,Exp,0.0,0.0,0.0,NaN,0.0,211.66,Velizy Grand Paris,25064.0,FR,Velizy,Grand Paris,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# data_frames = [customers, transactions]

# df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['CUSTOMER'],
#                                             how='outer'), data_frames)

In [ ]:
# missing

,MO_ID,SO_ID,CUSTOMER,END_CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,MO_CREATED_DATE,SO_CREATED_DATE,TECH,OFFER_TYPE,BUSINESS_TYPE,COSTS_PRODUCT_A,COSTS_PRODUCT_B,COSTS_PRODUCT_C,OFFER_STATUS,COSTS_PRODUCT_D,COSTS_PRODUCT_E,SALES_LOCATION,TEST_SET_ID,COUNTRY_x,SALES_OFFICE,SALES_BRANCH,Country,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,COUNTRY_y,CURRENCY
2,a050N000013fnwdQAA,a030N00001Eod9yQAB,3_Switzerland,NaN,6264.70,2296.00,1722.0,2168.56,SFT Standard,6820.0,2019,2019,F,FIR,E,0.0,0.00,0.0,WIN,0.00,0.0,Luzern Central,NaN,CH,Luzern,Branch Central,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,a050N000013fLN9QAM,a030N00001EoJN5QAN,17_Switzerland,NaN,5605.00,2480.00,1968.0,1513.70,SFT Standard,4763.0,2019,2019,F,FIR,M,0.0,0.00,0.0,Win,0.00,0.0,Geneva West,NaN,CH,Geneva,Branch West,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,a050N000013fNDYQA2,a030N00001EoKjdQAF,19_Switzerland,NaN,1075.50,775.00,615.0,62.14,SFT Standard,8299.0,2019,2019,F,FIR,E,0.0,0.00,0.0,Win,0.00,0.0,Lausanne West,NaN,CH,Lausanne,Branch West,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,a050N000013fNjeQAE,a030N00001EoLRQQA3,21_Switzerland,NaN,24330.00,14200.00,11316.0,6797.21,SFT Standard,0.0,2019,2019,S,IN,M,0.0,3329.29,0.0,WON,0.00,0.0,St. Blaise West,NaN,CH,St. Blaise,Branch West,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,a050N000013fQHYQA2,a030N00001EoPLjQAN,22_Switzerland,NaN,23414.90,0.00,0.0,13377.71,CMT Installer,4321.0,2019,2019,BP,GAM,N,0.0,0.00,0.0,Lose,0.00,0.0,Steinhausen Central,NaN,CH,Steinhausen,Branch Central,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26108,a055p00001OS0wJAAT,a035p00001QZSvdAAH,NA_France,NaN,24000.00,16668.67,13459.8,5600.47,Tarif public,0.0,2021,2021,F,FDD,New,0.0,0.00,0.0,LOsT,728.00,0.0,Aix Sud-Est,NaN,FR,Aix,Sud-Est,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26113,a055p00001OS4CQAA1,a035p00001QZVaTAAX,NA_France,NaN,5946.84,3375.00,2464.8,1424.50,Tarif public,1811.0,2021,2021,S,SU,New,0.0,0.00,0.0,Lose,0.00,0.0,Bezons Grand Paris,NaN,FR,Bezons,Grand Paris,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26115,a055p00001OSk9LAAT,a035p00001QZwQMAA1,NA_France,NaN,25000.00,13122.68,12843.0,4011.37,Tarif public,NaN,2021,2021,S,SU,Exp,0.0,0.00,0.0,Won,0.00,0.0,Strasbourg Grand Est,NaN,FR,Strasbourg,Grand Est,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26116,a055p00001OS6DMAA1,a035p00001QZXjKAAX,NA_France,NaN,11600.00,2740.00,1904.4,9101.91,Tarif public,0.0,2021,2021,F,FDI,New,0.0,0.00,0.0,Lost,2344.06,0.0,Metz Grand Est,NaN,FR,Metz,Grand Est,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_merged.shape

(26151, 34)

In [ ]:
# df_merged['MO_CREATED_DATE'] = df_merged['MO_CREATED_DATE'].str.slice(6, 10)
# df_merged['SO_CREATED_DATE'] = df_merged['SO_CREATED_DATE'].str.slice(6, 10)
# df_merged['CREATION_YEAR'] = df_merged['CREATION_YEAR'].str.slice(6, 10)

In [ ]:
df_merged['REV_CURRENT_YEAR']  = df_merged['REV_CURRENT_YEAR'].str.replace('"', '')

In [ ]:
df_merged.head(5)

,MO_ID,SO_ID,CUSTOMER,END_CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,MO_CREATED_DATE,SO_CREATED_DATE,TECH,OFFER_TYPE,BUSINESS_TYPE,COSTS_PRODUCT_A,COSTS_PRODUCT_B,COSTS_PRODUCT_C,OFFER_STATUS,COSTS_PRODUCT_D,COSTS_PRODUCT_E,SALES_LOCATION,TEST_SET_ID,COUNTRY_x,SALES_OFFICE,SALES_BRANCH,Country,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,COUNTRY_y,CURRENCY
0,a050N000013fnfrQAA,a030N00001EochoQAB,1_Switzerland,NaN,1711.0,1395.0,1107.0,186.30,SFT Standard,2100.0,2019,2019,S,IN,E,0.0,59.48,0.00,LOsT,0.0,0.0,Luzern Central,NaN,CH,Luzern,Branch Central,Switzerland,81283.9230769231,81283.923077,32203.615385,2004.0,Privately Owned/Publicly Traded,Switzerland,Chinese Yuan
1,a050N000013fgL1QAI,a030N00001EociNQAR,2_Switzerland,NaN,26687.6,14651.0,9282.0,7768.34,CMT Installer,7110.0,2019,2019,C,D,N,0.0,0.00,0.00,Lost,0.0,0.0,Zürich East,NaN,CH,Zürich,Branch East,Switzerland,0,0.000000,0.000000,2004.0,Privately Owned/Publicly Traded,Switzerland,Chinese Yuan
2,a050N000013fnwdQAA,a030N00001Eod9yQAB,3_Switzerland,NaN,6264.7,2296.0,1722.0,2168.56,SFT Standard,6820.0,2019,2019,F,FIR,E,0.0,0.00,0.00,WIN,0.0,0.0,Luzern Central,NaN,CH,Luzern,Branch Central,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a050N000013foAGQAY,a030N00001EodLEQAZ,4_Switzerland,4,4300.2,310.0,246.0,2775.92,SFT Standard,3821.0,2019,2019,F,FIR,M,0.0,0.00,0.00,Win,0.0,0.0,Basel Central,NaN,CH,Basel,Branch Central,Switzerland,12668.84,12668.840000,32731.180000,2003.0,Privately Owned/Publicly Traded,Switzerland,Euro
4,a050N000013foKVQAY,a030N00001EoeBiQAJ,5_Switzerland,NaN,13693.0,5815.0,4674.0,4179.38,SFT Standard,4719.0,2019,2019,F,FIR,E,0.0,0.00,1854.01,WIN,0.0,0.0,Geneva West,NaN,CH,Geneva,Branch West,Switzerland,7130.98,7130.980000,10210.860000,2004.0,Privately Owned/Publicly Traded,Switzerland,Euro


In [ ]:
df_merged = df_merged.drop(['MO_ID', 
                            'END_CUSTOMER',
                            'COUNTRY_x',
                            'SALES_OFFICE',
                            'SALES_BRANCH',
                            'COUNTRY_y',
                            'SALES_LOCATION'], axis = 1)

In [ ]:
df_merged['OFFER_STATUS'].unique()

array(['LOsT', 'Lost', 'WIN', 'Win', nan, 'Won', 'LOST', 'Lose', 'WON'],
      dtype=object)

In [ ]:
Win = ['WIN', 'Won', 'WON']
Lose = ['Lost', 'LOST', 'LOsT']

In [ ]:
df_merged['OFFER_STATUS'] = np.where(df_merged['OFFER_STATUS'].isin(Lose),
                                     'Lose', df_merged['OFFER_STATUS'])
df_merged['OFFER_STATUS'] = np.where(df_merged['OFFER_STATUS'].isin(Win),
                                     'Win', df_merged['OFFER_STATUS'])

In [ ]:
df_merged['OFFER_STATUS'].unique()

array(['Lose', 'Win', nan], dtype=object)

In [ ]:
df_merged.head(2)

,SO_ID,CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,MO_CREATED_DATE,SO_CREATED_DATE,TECH,OFFER_TYPE,BUSINESS_TYPE,COSTS_PRODUCT_A,COSTS_PRODUCT_B,COSTS_PRODUCT_C,OFFER_STATUS,COSTS_PRODUCT_D,COSTS_PRODUCT_E,TEST_SET_ID,Country,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,CURRENCY
0,a030N00001EochoQAB,1_Switzerland,1711.0,1395.0,1107.0,186.30,SFT Standard,2100.0,2019,2019,S,IN,E,0.0,59.48,0.0,Lose,0.0,0.0,NaN,Switzerland,81283.9230769231,81283.923077,32203.615385,2004.0,Privately Owned/Publicly Traded,Chinese Yuan
1,a030N00001EociNQAR,2_Switzerland,26687.6,14651.0,9282.0,7768.34,CMT Installer,7110.0,2019,2019,C,D,N,0.0,0.00,0.0,Lose,0.0,0.0,NaN,Switzerland,0,0.000000,0.000000,2004.0,Privately Owned/Publicly Traded,Chinese Yuan


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
labelEcoder_df = LabelEncoder()

In [ ]:
df_merged['OWNERSHIP'].unique()

array(['Privately Owned/Publicly Traded', nan, 'Governmental',
       'No information', 'Individual Person'], dtype=object)

In [ ]:
df_merged['PRICE_LIST'] = labelEcoder_df.fit_transform(df_merged['PRICE_LIST'])
df_merged['TECH'] = labelEcoder_df.fit_transform(df_merged['TECH'])
df_merged['OFFER_TYPE'] = labelEcoder_df.fit_transform(df_merged['OFFER_TYPE'])
df_merged['BUSINESS_TYPE'] = labelEcoder_df.fit_transform(df_merged['BUSINESS_TYPE'])
df_merged['Country'] = labelEcoder_df.fit_transform(df_merged['Country'])
df_merged['OWNERSHIP'] = labelEcoder_df.fit_transform(df_merged['OWNERSHIP'])
df_merged['ISIC'] = df_merged['ISIC'].apply(str)
df_merged['ISIC'] = labelEcoder_df.fit_transform(df_merged['ISIC'])

In [ ]:
df_merged.head(5)

,SO_ID,CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,MO_CREATED_DATE,SO_CREATED_DATE,TECH,OFFER_TYPE,BUSINESS_TYPE,COSTS_PRODUCT_A,COSTS_PRODUCT_B,COSTS_PRODUCT_C,OFFER_STATUS,COSTS_PRODUCT_D,COSTS_PRODUCT_E,TEST_SET_ID,Country,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,CURRENCY
0,a030N00001EochoQAB,1_Switzerland,1711.0,1395.0,1107.0,186.30,2,53,2019,2019,6,22,1,0.0,59.48,0.00,Lose,0.0,0.0,NaN,1,81283.9230769231,81283.923077,32203.615385,2004.0,3,Chinese Yuan
1,a030N00001EociNQAR,2_Switzerland,26687.6,14651.0,9282.0,7768.34,1,248,2019,2019,1,6,6,0.0,0.00,0.00,Lose,0.0,0.0,NaN,1,0,0.000000,0.000000,2004.0,3,Chinese Yuan
2,a030N00001Eod9yQAB,3_Switzerland,6264.7,2296.0,1722.0,2168.56,2,243,2019,2019,4,19,1,0.0,0.00,0.00,Win,0.0,0.0,NaN,1,NaN,NaN,NaN,NaN,4,NaN
3,a030N00001EodLEQAZ,4_Switzerland,4300.2,310.0,246.0,2775.92,2,133,2019,2019,4,19,4,0.0,0.00,0.00,Win,0.0,0.0,NaN,1,12668.84,12668.840000,32731.180000,2003.0,3,Euro
4,a030N00001EoeBiQAJ,5_Switzerland,13693.0,5815.0,4674.0,4179.38,2,165,2019,2019,4,19,1,0.0,0.00,1854.01,Win,0.0,0.0,NaN,1,7130.98,7130.980000,10210.860000,2004.0,3,Euro


In [ ]:
# df_merged['REV_CURRENT_YEAR'] = np.where(df_merged['CURRENCY'] == 'Chinese Yuan',
#                                          df_merged['REV_CURRENT_YEAR']*0.14,
#                                          np.where(df_merged['CURRENCY'] == 'Pound Sterling',
#                                                   df_merged['REV_CURRENT_YEAR']*1.20,
#                                                   np.where(df_merged['CURRENCY'] == 'US Dollar',
#                                                            df_merged['REV_CURRENT_YEAR']*0.88,
#                                                            df_merged['REV_CURRENT_YEAR'])))

In [ ]:
df_merged['REV_CURRENT_YEAR.1'] = np.where(df_merged['CURRENCY'] == 'Chinese Yuan',
                                         df_merged['REV_CURRENT_YEAR.1']*0.14,
                                         np.where(df_merged['CURRENCY'] == 'Pound Sterling',
                                                  df_merged['REV_CURRENT_YEAR.1']*1.20,
                                                  np.where(df_merged['CURRENCY'] == 'US Dollar',
                                                           df_merged['REV_CURRENT_YEAR.1']*0.88,
                                                           df_merged['REV_CURRENT_YEAR.1'])))

In [ ]:
# df_merged['REV_CURRENT_YEAR.2'] = np.where(df_merged['CURRENCY'] == 'Chinese Yuan',
#                                          df_merged['REV_CURRENT_YEAR.2']*0.14,
#                                          np.where(df_merged['CURRENCY'] == 'Pound Sterling',
#                                                   df_merged['REV_CURRENT_YEAR.2']*1.20,
#                                                   np.where(df_merged['CURRENCY'] == 'US Dollar',
#                                                            df_merged['REV_CURRENT_YEAR.2']*0.88,
#                                                            df_merged['REV_CURRENT_YEAR.2'])))

In [ ]:
train = df_merged[df_merged['TEST_SET_ID'].isna()]
test  = df_merged[df_merged['TEST_SET_ID'].notnull()]

In [ ]:
len(test)

2576

In [ ]:
missing_test = test[test['REV_CURRENT_YEAR'].isnull()]

In [ ]:
missing_train = train[train['REV_CURRENT_YEAR'].isnull()]

In [ ]:
len(train)-len(missing_train)

20768

In [ ]:
len(missing_train)/len(train)

0.11906680805938494

In [ ]:
len(missing_test)/len(test)

0.03687888198757764

In [ ]:
train = train.drop(['TEST_SET_ID', 'CUSTOMER'], axis = 1)
test = test.drop(['TEST_SET_ID', 'CUSTOMER'], axis = 1)

In [ ]:
 train = train.drop(['CURRENCY'], axis=1)

In [ ]:
#encode prediction variable 'OFFER_STATUS': Win=1, Loss=0
train['OFFER_STATUS'] = np.where(train['OFFER_STATUS'].str.contains("Win"), 1, 0)

In [ ]:
train.isnull().sum()

SO_ID                    0
OFFER_PRICE              0
SERVICE_LIST_PRICE       0
MATERIAL_COST            0
SERVICE_COST             0
PRICE_LIST               0
ISIC                     0
MO_CREATED_DATE          0
SO_CREATED_DATE          0
TECH                     0
OFFER_TYPE               0
BUSINESS_TYPE            0
COSTS_PRODUCT_A          0
COSTS_PRODUCT_B          0
COSTS_PRODUCT_C          0
OFFER_STATUS             0
COSTS_PRODUCT_D          0
COSTS_PRODUCT_E          0
Country                  0
REV_CURRENT_YEAR      2807
REV_CURRENT_YEAR.1    2807
REV_CURRENT_YEAR.2    2807
CREATION_YEAR         2807
OWNERSHIP                0
dtype: int64

In [ ]:
# train['OFFER_STATUS'].isnull().sum().sum()

0

In [ ]:
#df_merged.drop(['CREATION_YEAR', 'MO_CREATED_DATE', 'SO_CREATED_DATE'], axis=1, inplace=True)

In [ ]:
#df_merged['REV_CURRENT_YEAR']  = df_merged['REV_CURRENT_YEAR'].str.replace('"', '')

In [ ]:
#df_merged['REV_CURRENT_YEAR']  = pd.to_numeric(df_merged['REV_CURRENT_YEAR'])

In [ ]:
#df_merged.head(2)

,SO_ID,CUSTOMER,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,MO_CREATED_DATE,SO_CREATED_DATE,TECH,OFFER_TYPE,BUSINESS_TYPE,COSTS_PRODUCT_A,COSTS_PRODUCT_B,COSTS_PRODUCT_C,OFFER_STATUS,COSTS_PRODUCT_D,COSTS_PRODUCT_E,SALES_LOCATION,TEST_SET_ID,Country,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,CREATION_YEAR,OWNERSHIP,CURRENCY
0,a030N00001EochoQAB,1_Switzerland,0.013167,0.022929,0.025000,0.024898,SFT Standard,2100.0,2019,2019,S,IN,E,0.0,0.076119,0.0,Lose,0.0,0.0,Luzern Central,NaN,Switzerland,81283.9230769231,0.003092,0.001402,2004,Privately Owned/Publicly Traded,Chinese Yuan
1,a030N00001EociNQAR,2_Switzerland,0.475696,0.240812,0.209621,0.142824,CMT Installer,7110.0,2019,2019,C,D,N,0.0,0.073921,0.0,Lose,0.0,0.0,Zürich East,NaN,Switzerland,0,0.001202,0.000910,2004,Privately Owned/Publicly Traded,Chinese Yuan


In [ ]:
#df_merged['END_CUSTOMER'].isna().sum()

33091

In [ ]:
#df_merged.drop(['END_CUSTOMER'], axis=1, inplace=True)

In [ ]:
#geo.head(15)

,COUNTRY,SALES_OFFICE,SALES_BRANCH,SALES_LOCATION
0,CH,Luzern,Branch Central,Luzern Central
1,CH,Zürich,Branch East,Zürich East
2,CH,Basel,Branch Central,Basel Central
3,CH,Geneva,Branch West,Geneva West
4,CH,Lausanne,Branch West,Lausanne West
5,CH,Sion,Branch West,Sion West
6,CH,St. Gallen,Branch East,St. Gallen East
7,CH,St. Blaise,Branch West,St. Blaise West
8,CH,Steinhausen,Branch Central,Steinhausen Central
9,CH,Bern,Branch Central,Bern Central


In [ ]:
# data_frames = [df_merged, geo]

# df_all = reduce(lambda  left,right: pd.merge(left,right,on=['COUNTRY', 'SALES_LOCATION'],
#                                              how='outer'), data_frames)

In [ ]:
#df_all.head(15)

,CUSTOMER,REV_CURRENT_YEAR,REV_CURRENT_YEAR.1,REV_CURRENT_YEAR.2,OWNERSHIP,COUNTRY,CURRENCY,MO_ID,SO_ID,OFFER_PRICE,SERVICE_LIST_PRICE,MATERIAL_COST,SERVICE_COST,PRICE_LIST,ISIC,TECH,OFFER_TYPE,BUSINESS_TYPE,COSTS_PRODUCT_A,COSTS_PRODUCT_B,COSTS_PRODUCT_C,OFFER_STATUS,COSTS_PRODUCT_D,COSTS_PRODUCT_E,SALES_LOCATION,TEST_SET_ID
0,579.0,157832.813953,157832.813953,174585.011628,Privately Owned/Publicly Traded,Switzerland,US Dollar,a050N00001FgwyGQAR,a030N00001JWuAMQA1,3074.00,1520.0,1230.0,968.52,SFT Standard,2630.0,S,A,N,0.0,0.0,0.0,LOsT,0.0,0.0,Lausanne West,NaN
1,579.0,157832.813953,157832.813953,174585.011628,Privately Owned/Publicly Traded,Switzerland,US Dollar,a050N00001Cu5a6QAB,a030N00001I2pOnQAJ,7357.25,2170.0,1722.0,2677.00,SFT Standard,2630.0,S,A,N,0.0,0.0,0.0,WIN,0.0,0.0,Lausanne West,NaN
2,579.0,157832.813953,157832.813953,174585.011628,Privately Owned/Publicly Traded,Switzerland,US Dollar,a050N00001CursyQAB,a030N00001I2zdLQAR,7357.25,2170.0,1722.0,2677.00,SFT Standard,2630.0,S,A,N,0.0,0.0,0.0,WON,0.0,0.0,Lausanne West,NaN
3,585.0,291167.521368,291167.521368,204828.512821,Privately Owned/Publicly Traded,Switzerland,Pound Sterling,a050N00001HTKDkQAP,a030N00001LyeQcQAJ,1368.90,1437.0,1107.0,0.00,SFT Standard,1200.0,F,FIR,E,369.0,0.0,0.0,WON,0.0,0.0,Lausanne West,NaN
4,585.0,291167.521368,291167.521368,204828.512821,Privately Owned/Publicly Traded,Switzerland,Pound Sterling,a050N00001M60DpQAJ,a030N00001OPOxPQAX,2587.50,358.0,232.0,1772.96,SFT Standard,1200.0,F,GA,M,0.0,0.0,0.0,LOsT,0.0,0.0,Lausanne West,NaN
5,404.0,14575.639535,14575.639535,32455.941860,Privately Owned/Publicly Traded,Switzerland,US Dollar,a050N00001EWdqwQAD,a030N00001JUvT7QAL,6963.00,2790.0,2214.0,2068.64,SFT Standard,8110.0,S,A,M,0.0,0.0,0.0,Lost,0.0,0.0,Lausanne West,NaN
6,404.0,14575.639535,14575.639535,32455.941860,Privately Owned/Publicly Traded,Switzerland,US Dollar,a050N00001EWbM5QAL,a030N00001JUu9FQAT,9233.00,3100.0,2460.0,3314.60,SFT Standard,8110.0,S,A,M,0.0,0.0,0.0,LOsT,0.0,0.0,Lausanne West,NaN
7,254.0,93863.950000,93863.950000,58463.670000,Privately Owned/Publicly Traded,Switzerland,Euro,a050N00001EYkTLQA1,a030N00001JVmpHQAT,2917.00,1520.0,1230.0,1100.21,SFT Standard,2652.0,F,GA,E,0.0,0.0,0.0,Lose,0.0,0.0,Lausanne West,NaN
8,254.0,93863.950000,93863.950000,58463.670000,Privately Owned/Publicly Traded,Switzerland,Euro,a050N00001EYkjTQAT,a030N00001JVmw8QAD,2993.00,1070.0,861.0,1100.21,SFT Standard,2652.0,F,GA,E,0.0,0.0,0.0,LOST,0.0,0.0,Lausanne West,NaN
9,254.0,93863.950000,93863.950000,58463.670000,Privately Owned/Publicly Traded,Switzerland,Euro,a050N00001M76xZQAR,a030N00001OPg8IQAT,11089.35,5091.0,3364.0,5411.35,SFT Standard,2652.0,F,GA,N,0.0,0.0,0.0,Won,0.0,0.0,Lausanne West,NaN


In [ ]:
#print(df_all['SALES_BRANCH'].isna().sum())
#print(df_all['SALES_OFFICE'].isna().sum())

43642
43643


In [ ]:
#mit transactions und customer
#df_merged.shape

(43641, 26)

In [ ]:
#mit transactions, customer und geo
#df_all.shape

(43687, 28)

In [ ]:
#df_all.drop(['SALES_BRANCH', 'SALES_OFFICE'], axis=1, inplace=True)

In [ ]:
#print(df_all.apply(lambda col: col.unique()))

CUSTOMER              [579.0, 585.0, 404.0, 254.0, 727.0, 3626.0, 17...
REV_CURRENT_YEAR      [157832.81395348802, 291167.521367521, 14575.6...
REV_CURRENT_YEAR.1    [157832.81395348837, 291167.52136752143, 14575...
REV_CURRENT_YEAR.2    [174585.01162790696, 204828.5128205128, 32455....
OWNERSHIP             [Privately Owned/Publicly Traded, Governmental...
COUNTRY                              [Switzerland, France, nan, CH, FR]
CURRENCY              [US Dollar, Pound Sterling, Euro, Chinese Yuan...
MO_ID                 [a050N00001FgwyGQAR, a050N00001Cu5a6QAB, a050N...
SO_ID                 [a030N00001JWuAMQA1, a030N00001I2pOnQAJ, a030N...
OFFER_PRICE           [3074.0, 7357.25, 1368.9, 2587.5, 6963.0, 9233...
SERVICE_LIST_PRICE    [1520.0, 2170.0, 1437.0, 358.0, 2790.0, 3100.0...
MATERIAL_COST         [1230.0, 1722.0, 1107.0, 232.0, 2214.0, 2460.0...
SERVICE_COST          [968.52, 2677.0, 0.0, 1772.96, 2068.64, 3314.6...
PRICE_LIST            [SFT Standard, CMT End Customer, CMT Insta

In [ ]:
#df_all['OWNERSHIP'].unique()

array(['Privately Owned/Publicly Traded', 'Governmental',
       'Individual Person', 'No information', nan], dtype=object)

In [ ]:
#df_all['SO_ID'].nunique()

24788

In [ ]:
#df_all['MO_ID'].nunique()

24711